## Importing required libraries

In [1]:
from tqdm import trange
import numpy as np
import cv2
from scipy.ndimage import convolve
import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")

In [2]:
def energyCalc(img):
    
    energyMap = cv2.imread(energyMapFile,-1)

    return energyMap

## Function to Find Seam with Minimum Energy

In [3]:
def minEnergySeam(img):
    r, c, _ = img.shape
    energyMap = energyCalc(img)
    M = energyMap.copy()
    backtrack = np.zeros_like(M, dtype=np.int)
    for i in range(1, r):
        for j in range(0, c):
            if j == 0:
                idx = np.argmin(M[i-1, j:j + 2])
                backtrack[i, j] = idx + j
                minEnergy = M[i-1, idx + j]
            else:
                idx = np.argmin(M[i - 1, j - 1:j + 2])
                backtrack[i, j] = idx + j - 1
                minEnergy = M[i - 1, idx + j - 1]

            M[i, j] += minEnergy

    return M, backtrack

## Function for Deleting Pixels from Seam with Minimum Energy

In [4]:
def carveVert(img):
    r, c, _ = img.shape
    M, backtrack = minEnergySeam(img)
    mask = np.ones((r, c), dtype=np.bool)
    j = np.argmin(M[-1])
    for i in reversed(range(r)):
        mask[i, j] = False
        j = backtrack[i, j]
    maskEnergy=mask
    energyMap=energyCalc(img)
    mask = np.stack([mask] * 3, axis=2)
    img = img[mask].reshape((r, c - 1, 3))
    energyMap = energyMap[maskEnergy].reshape((r, c - 1, 1))
    cv2.imwrite(energyMapFile, energyMap)
    return img

## Function for row-wise carving

In [5]:
def rCrop(img, rScale):
    energyMap=energyCalc(img)
    img = np.rot90(img, 1, (0, 1))
    energyMap = np.rot90(energyMap, 1, (0, 1))
    cv2.imwrite(energyMapFile, energyMap)
    img = cCrop(img, rScale)
    energyMap = cv2.imread(energyMapFile,-1)
    img = np.rot90(img, 3, (0, 1))
    energyMap = np.rot90(energyMap, 3, (0, 1))
    cv2.imwrite(energyMapFile, energyMap)
    return img

## Function for column-wise carving

In [6]:
def cCrop(img, cScale):
    r, c, _ = img.shape
    cNew = int(cScale * c)
    for i in trange(c - cNew):
        img = carveVert(img)
    return img

## Call cell

In [ ]:
scale = 0.8

inFile = "./input.png"
outFile = "./output.png"
energyMapFile = "./energy_map.png"
energyMapInitial = "./energy_map_initial.png"

model = tf.keras.models.load_model("./model_joined_generator")

inp = cv2.imread(inFile)
ht, wd, channels = inp.shape

img = tf.keras.utils.load_img(inFile,target_size=(256,256))
img = tf.keras.utils.img_to_array(img)
img = tf.expand_dims(img,0)

energyMap = model.predict(img)
energyMap = np.reshape(energyMap,(256,256))*255
energyMap = cv2.resize(energyMap,(wd,ht))
energyMap = energyMap.astype(np.uint8)

cv2.imwrite(energyMapInitial, energyMap)
cv2.imwrite(energyMapFile, energyMap)

outR = rCrop(inp, scale)
out = cCrop(outR, scale)
cv2.imwrite(outFile, out)

 80%|█████████████████████████████████████████████████████████████████▌                | 52/65 [01:01<00:13,  1.07s/it]